In [ ]:
import pandas as pd
import requests
import io
import os # Import the os module
from datetime import datetime

def update_results_csv(folder_name='football_data', filename='english_football_results_all_time.csv'):
    """
    Updates an existing CSV file in a local folder with the latest results.
    
    Args:
        folder_name (str): The name of the folder where the CSV is stored.
        filename (str): The name of the CSV file to update.
    """
    
    file_path = os.path.join(folder_name, filename)

    try:
        # Load the existing data from the folder
        existing_df = pd.read_csv(file_path)
        print(f"Successfully loaded existing data from {file_path}.")
        
        # Get the most recent match date and division
        existing_df['MatchDate'] = pd.to_datetime(existing_df['MatchDate'])
        most_recent_date = existing_df['MatchDate'].max()
        
        # The last division in the file is the one we'll update
        division_to_update = existing_df['Division'].iloc[-1]
        
        # Get the current season
        current_year = datetime.now().year
        current_month = datetime.now().month
        if current_month < 8:
            season = f'{current_year-1}{str(current_year)[2:]}'
        else:
            season = f'{current_year}{str(current_year+1)[2:]}'
            
        url = f'https://www.football-data.co.uk/mmz4281/{season}/{division_to_update}.csv'
        
        try:
            response = requests.get(url)
            response.raise_for_status()
            
            new_df = pd.read_csv(io.StringIO(response.text), encoding='latin-1')
            new_df.rename(columns={'Div': 'Division', 'Date': 'MatchDate'}, inplace=True)
            new_df['MatchDate'] = pd.to_datetime(new_df['MatchDate'], format='%d/%m/%y', errors='coerce')
            new_df['Season'] = season
            new_df['Division'] = division_to_update
            
            # Filter for new results since the last update
            new_results = new_df[new_df['MatchDate'] > most_recent_date]
            
            if not new_results.empty:
                updated_df = pd.concat([existing_df, new_results], ignore_index=True)
                updated_df.to_csv(file_path, index=False)
                print(f"Successfully appended {len(new_results)} new results to {file_path}. ✅")
            else:
                print("No new results to add. The CSV is already up to date. 👍")

        except requests.exceptions.RequestException as e:
            print(f'Error downloading data for update: {e}')
            print('Could not update the CSV file.')
            
    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found.")
        print("Please run the initial download script first.")

# Example usage:
if __name__ == '__main__':
    update_results_csv()